In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import matplotlib.pyplot as plt
import tabulate as tb

In [ ]:
TICKER = "AAPL"
START_DATE = "2000-01-01"
END_DATE = None
TIMEFRAME = "1d"    # 1d, 1h, 15m, 5m

In [3]:
df = yf.download(
    TICKER, 
    start=START_DATE,
    end=END_DATE,
    interval=TIMEFRAME,
    progress=False
)

if isinstance(df.columns, pd.MultiIndex):
    df.columns = [col[0] for col in df.columns]

df.dropna(inplace=True)
print(tb.tabulate(df.tail(), headers='keys', tablefmt='psql'))

C:\Users\dzied\AppData\Local\Temp\ipykernel_18168\3251543518.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(


+---------------------+---------+--------+--------+--------+-------------+
| Date                |   Close |   High |    Low |   Open |      Volume |
|---------------------+---------+--------+--------+--------+-------------|
| 2025-10-28 00:00:00 |  269    | 269.89 | 268.15 | 268.99 | 4.15348e+07 |
| 2025-10-29 00:00:00 |  269.7  | 271.41 | 267.11 | 269.28 | 5.10867e+07 |
| 2025-10-30 00:00:00 |  271.4  | 274.14 | 268.48 | 271.99 | 6.98865e+07 |
| 2025-10-31 00:00:00 |  270.37 | 277.32 | 269.16 | 276.99 | 8.61671e+07 |
| 2025-11-03 00:00:00 |  269.05 | 270.85 | 266.25 | 270.42 | 5.01387e+07 |
+---------------------+---------+--------+--------+--------+-------------+


In [4]:
df["rsi"] = ta.rsi(df["Close"], length=14)
macd = ta.macd(df["Close"])
df = df.join(macd)

bbands = ta.bbands(df["Close"], length=20)
df = df.join(bbands)

df["ema_10"] = ta.ema(df["Close"], length=10)
df["ema_20"] = ta.ema(df["Close"], length=20)
df["ema_50"] = ta.ema(df["Close"], length=50)
df["atr"] = ta.atr(df["High"], df["Low"], df["Close"], length=14)
stoch = ta.stoch(df["High"], df["Low"], df["Close"])
df["stoch_k"] = stoch.iloc[:, 0]
df["stoch_d"] = stoch.iloc[:, 1]
df["returns"] = df["Close"].pct_change()
df['direction'] = np.where(df['returns'] > 0, 1, 0)

df.dropna(inplace=True)

print(tb.tabulate(df.head(), headers='keys', tablefmt='psql'))
print(tb.tabulate(df.tail(), headers='keys', tablefmt='psql'))

df.to_csv(f"./../data/{TICKER}.csv")

+---------------------+----------+----------+----------+----------+-------------+---------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+----------+----------+----------+-----------+-----------+-----------+------------+-------------+
| Date                |    Close |     High |      Low |     Open |      Volume |     rsi |   MACD_12_26_9 |   MACDh_12_26_9 |   MACDs_12_26_9 |   BBL_20_2.0_2.0 |   BBM_20_2.0_2.0 |   BBU_20_2.0_2.0 |   BBB_20_2.0_2.0 |   BBP_20_2.0_2.0 |   ema_10 |   ema_20 |   ema_50 |       atr |   stoch_k |   stoch_d |    returns |   direction |
|---------------------+----------+----------+----------+----------+-------------+---------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+----------+----------+----------+-----------+-----------+-----------+------------+-------